# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,36.014,67,100,1.54,US,1706648118
1,1,port mathurin,-19.6833,63.4167,81.446,77,100,8.93,MU,1706648118
2,2,ushuaia,-54.8000,-68.3000,60.458,55,75,3.60,AR,1706648118
3,3,lichinga,-13.3128,35.2406,68.090,83,40,3.09,MZ,1706648118
4,4,puerto ayora,-0.7393,-90.3518,87.764,78,87,5.25,EC,1706648118


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Config the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
#     scale = 0.01,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

weather_df_ini = city_data_df.loc[(city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4.5)]
weather_df_ini

# weather_df = weather_df_ini.loc[(weather_df_ini["Max Temp"] > 21) & (weather_df_ini["Max Temp"] < 27)]
# weather_df #there are no records with this Max Temp criteria, so let's change the criteria for Max Temp
weather_df = weather_df_ini.loc[(weather_df_ini["Max Temp"] > 50) & (weather_df_ini["Max Temp"] <= 60)]

#Drop any rows with null values
weather_df.dropna()

#Display sample data
weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,fort bragg,35.1390,-79.0060,56.822,46,0,2.57,US,1706648125
106,106,hassi messaoud,31.6804,6.0729,57.056,44,0,2.57,DZ,1706648118
148,148,port pirie,-33.1833,138.0167,58.802,65,0,4.01,AU,1706647982
167,167,anacapri,40.5558,14.2136,53.132,66,0,0.00,IT,1706648146
205,205,greenville,35.6127,-77.3663,53.762,50,0,2.06,US,1706648137
272,272,tura,25.5198,90.2201,55.472,50,0,1.65,IN,1706648162
332,332,tamanrasset,22.7850,5.5228,55.310,33,0,1.03,DZ,1706648172
341,341,laramie,41.3114,-105.5911,54.950,34,0,1.54,US,1706648173
367,367,myitkyina,25.3833,97.4000,52.610,68,0,1.31,MM,1706648177
380,380,juhaynah,26.6732,31.4976,50.324,42,0,3.21,EG,1706648179


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
37,fort bragg,US,35.1390,-79.0060,46,
106,hassi messaoud,DZ,31.6804,6.0729,44,
148,port pirie,AU,-33.1833,138.0167,65,
167,anacapri,IT,40.5558,14.2136,66,
205,greenville,US,35.6127,-77.3663,50,
272,tura,IN,25.5198,90.2201,50,
332,tamanrasset,DZ,22.7850,5.5228,33,
341,laramie,US,41.3114,-105.5911,34,
367,myitkyina,MM,25.3833,97.4000,68,
380,juhaynah,EG,26.6732,31.4976,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "radius": 10000,
    "types": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
#     get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = lat
    params["bias"] = lng
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()

#     Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: No hotel found
hassi messaoud - nearest hotel: No hotel found
port pirie - nearest hotel: No hotel found
anacapri - nearest hotel: No hotel found
greenville - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
laramie - nearest hotel: No hotel found
myitkyina - nearest hotel: No hotel found
juhaynah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
37,fort bragg,US,35.1390,-79.0060,46,No hotel found
106,hassi messaoud,DZ,31.6804,6.0729,44,No hotel found
148,port pirie,AU,-33.1833,138.0167,65,No hotel found
167,anacapri,IT,40.5558,14.2136,66,No hotel found
205,greenville,US,35.6127,-77.3663,50,No hotel found
272,tura,IN,25.5198,90.2201,50,No hotel found
332,tamanrasset,DZ,22.7850,5.5228,33,No hotel found
341,laramie,US,41.3114,-105.5911,34,No hotel found
367,myitkyina,MM,25.3833,97.4000,68,No hotel found
380,juhaynah,EG,26.6732,31.4976,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
#     frame_width = 700,
#     frame_height = 500,
    size = "Humidity",
#     scale = 0.01,
    color = "City"
)

# Display figure
map_plot_2




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)